In [3]:
#%pip install --extra-index-url https://test.pypi.org/simple ../../EnergyPlus-OOEP/
#%pip install git+https://github.com/NTU-CCA-HVAC-OPTIM-a842a748/EnergyPlus-Datasets.git
#%pip install ipywidgets tqdm gymnasium torch ray[rllib]
#%pip install pythermalcomfort
#%pip install matplotlib
#%pip install matplotlib ipympl

In [4]:
import energyplus.ooep as _ooep_

from energyplus.ooep.addons.logging import ProgressLogger

from energyplus.ooep import (
    Simulator,
    InputModel,
    Report,
)

from energyplus.dataset.basic import dataset as _epds_

simulator = _ooep_.Simulator().add(
    ProgressLogger(),
)


ModuleNotFoundError: No module named 'energyplus.ooep.addons.display'

In [ ]:
model = InputModel().loadf('./data/model-current.idf')

EnergyPlus Starting
EnergyPlus, Version 23.2.0-7636e6b3e9, YMD=2024.04.21 01:42
Converted input file format. Exiting.
EnergyPlus Run Time=00hr 00min  0.22sec


EnergyPlus Completed Successfully.


In [ ]:
model['Timestep']['Timestep 1']['number_of_timesteps_per_hour'] = 2

In [ ]:
simulator.awaitable.run_forever(
    input=Simulator.InputSpecs(
        #model='./data/model-current.idf',
        model=model,
        weather=f'./data/SGP_Singapore_486980_IWEC.epw',
    ),
    output=Simulator.OutputSpecs(
        report=Report().open('.ooep-report-9e1287d2-8e75-4cf5-bbc5-f76580b56a69'),
    ),
    options=Simulator.RuntimeOptions(
        #design_day=True,
    ),
)

<Task pending name='Task-5' coro=<Engine.run_forever() running at /home/user@AD/lab/reports/apr01/.venv/lib/python3.11/site-packages/energyplus/ooep/utils/awaitables.py:68>>

In [ ]:
import numpy as _numpy_
import gymnasium as _gymnasium_
from energyplus.ooep.addons.rl.gymnasium.spaces import VariableBox
from energyplus.ooep.addons.rl.ray import SimulatorEnv

from energyplus.ooep.components.variables import (
    Actuator,
    OutputVariable,
)
import pythermalcomfort as pytc
import matplotlib as plt
from ray.rllib.algorithms.ppo import PPOConfig
import time
%config InlineBackend.figure_formats = ['svg']
%matplotlib widget

import matplotlib as _matplotlib_
import matplotlib.pyplot as _plt_



class RewardFunction:
    def __init__(self, metab_rate=1.5, clothing=.5, pmv_limit=3.):
        self._metab_rate = _numpy_.asarray(metab_rate)
        self._clothing = _numpy_.asarray(clothing)
        self._pmv_limit = _numpy_.asarray(pmv_limit)
        self.energy_consumption_largest = 0.1
   
    def __call__(self, observation):            
        self.energy_consumption = observation['AHU energy consumption']
        if self.energy_consumption >= self.energy_consumption_largest:
            self.energy_consumption_largest = self.energy_consumption
        self.energy_efficiency = (self.energy_consumption_largest - self.energy_consumption) / self.energy_consumption_largest
        pmv = pytc.models.pmv_ppd(
            tdb=(tdb := observation['temperature:drybulb']), 
            tr=observation['temperature:radiant'], 
            # calculate relative air speed
            vr=pytc.utilities.v_relative(v=observation.get('airspeed', .1), met=self._metab_rate), 
            rh=observation['humidity'], 
            met=self._metab_rate, 
            # calculate dynamic clothing
            clo=pytc.utilities.clo_dynamic(clo=self._clothing, met=self._metab_rate),
        )['pmv']
        #print(((self._pmv_limit - _numpy_.abs(pmv)) / self._pmv_limit) - self.energy_efficiency)
        detect= _numpy_.mean(
                [(self._pmv_limit - _numpy_.abs(pmv)) / self._pmv_limit, -self.energy_efficiency],
                axis=0,
            )
        # draw_curve.update(self.detect) 

        simulator.variables.on(_ooep_.WallClock.Ref())
        if _numpy_.isnan(detect):
            print(simulator.variables[_ooep_.WallClock.Ref()].value, observation)
            return 0.
        # print(simulator.variables[_ooep_.WallClock.Ref()].value)
        return (
            (self._pmv_limit - _numpy_.abs(pmv)) / self._pmv_limit
        )
    
config = (
    PPOConfig.from_dict({        
        'create_env_on_local_worker': True
    })
    .resources(num_gpus=1)
    .environment(
        SimulatorEnv, 
        env_config=SimulatorEnv.Config(
            action_space=_gymnasium_.spaces.Dict({
                'temperature': VariableBox(
                    low=20., high=30.,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(Actuator.Ref(
                    type='Zone Temperature Control',
                    control_type='Cooling Setpoint',
                    key='1FXOFFICE:ZONE1',
                )),
                'humidity':VariableBox(
                    low=0., high=1.,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(Actuator.Ref(
                    type='System Node Setpoint',
                    control_type='Humidity Ratio Setpoint',
                    key='1FXOFFICE:ZONE1 ZONE AIR NODE',
                )),
            }),    
            observation_space=_gymnasium_.spaces.Dict({
                'temperature:drybulb': VariableBox(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Mean Air Temperature',
                    key='1FXOFFICE:ZONE1',
                )),
                'temperature:radiant' : VariableBox(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Mean Radiant Temperature',
                    key='1FXOFFICE:ZONE1',
                )),
                'humidity' : VariableBox(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Air Relative Humidity',
                    key='1FXOFFICE:ZONE1',
                )),
                'AHU energy consumption' : VariableBox(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Fan Electricity Rate',
                    key='AIR LOOP 5 AHU SUPPLY FAN',
                )),
            }),
            reward_function=RewardFunction(),
            event_refs=[
                'begin_zone_timestep_after_init_heat_balance',
            ],
            simulator_factory=lambda simulator=simulator: simulator,
        )
    )
    .rollouts(
        #num_rollout_workers=10,
        num_rollout_workers=0,
        enable_connectors=False,
    )
    .framework("torch")
    .training(model={"fcnet_hiddens": [64, 64]},
             lr=0.0001)
    .evaluation(
        #evaluation_interval=1,
        #evaluation_num_workers=0
    )
)


algo = config.build(use_copy=True)

/home/user@AD/lab/reports/apr01/.venv/lib/python3.11/site-packages/matplotlib_inline/config.py:59: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):
/home/user@AD/lab/reports/apr01/.venv/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:500: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/user@AD/lab/reports/apr01/.venv/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in

In [ ]:
# force simulator to rerun to make the newly requested variables available
simulator.stop()
# start training
for _ in range(10):
    print(algo.train())

In [ ]:
algo.evaluate()

KeyboardInterrupt: 

In [ ]:
simulator.stop()